## Save and Load Regression Model
For more details https://learn.microsoft.com/en-us/dotnet/machine-learning/how-to-guides/save-load-machine-learning-models-ml-net

In [1]:
#r "nuget: Microsoft.ML, 5.0.0"
#r "nuget: Microsoft.ML.FastTree, 5.0.0"
#r "nuget: Microsoft.ML.LightGbm, 5.0.0"

using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Transforms;

Installed Packages Microsoft.ML, 5.0.0 Microsoft.ML.FastTree, 5.0.0 Microsoft.ML.LightGbm, 5.0.0

In [2]:
public class SalaryData
{
    [LoadColumn(0)]
    public float YearsExperience;

    [LoadColumn(1)]
    public float Salary;
}

public class SalaryPrediction
{
    [ColumnName("Score")]
    public float Salary;
}

In [3]:
MLContext mlContext = new MLContext();
IDataView data = mlContext.Data.LoadFromTextFile<SalaryData>(path: "Salary_data.csv", hasHeader: true, separatorChar: ',');

In [4]:

var pipeline =mlContext.Transforms.Concatenate("Features", nameof(SalaryData.YearsExperience))
.Append(mlContext.Transforms.NormalizeMeanVariance("Features"))
.Append(mlContext.Regression.Trainers.Sdca(labelColumnName: nameof(SalaryData.Salary), featureColumnName: "Features"));

In [5]:
// Split and train the model
var split = mlContext.Data.TrainTestSplit(data, testFraction: 0.2);

In [6]:
// Train the model
var model=pipeline.Fit(split.TrainSet);

In [7]:
// Test the model
var prediction=model.Transform(split.TestSet);

In [9]:
// Evualuate the model
var metrics=mlContext.Regression.Evaluate(prediction,labelColumnName:nameof(SalaryData.Salary),scoreColumnName:"Score");

In [13]:
Console.WriteLine("===== MODEL METRICS =====");
Console.WriteLine($"R²  : {metrics.RSquared}");
Console.WriteLine($"MAE: {metrics.MeanAbsoluteError}");
Console.WriteLine($"RMSE: {metrics.RootMeanSquaredError}");
Console.WriteLine($"MSE: {metrics.MeanSquaredError}");

===== MODEL METRICS =====
R²  : 0.987766441751266
MAE: 2581.85546875
RMSE: 2893.9750391655475
MSE: 8375091.527313232


In [14]:
// Save the Model in disk
mlContext.Model.Save(model, data.Schema, "SalaryPredictionModel.zip");

In [ ]:
// Load the model from disk
MLContext _mlcontext=new MLContext();
DataViewSchema modelInputSchema;

var loadedModel=mlContext.Model.Load("SalaryPredictionModel.zip",out modelInputSchema);

In [20]:
// Predict a sample
var predEngine=_mlcontext.Model.CreatePredictionEngine<SalaryData,SalaryPrediction>(loadedModel);
var sampleData=new SalaryData()
{
    YearsExperience=4.0F,Salary=0F
};

var result=predEngine.Predict(sampleData);
Console.WriteLine($"Predicted Salary for {sampleData.YearsExperience} years of experience is {result.Salary}");

Predicted Salary for 4 years of experience is 67860.86
